## Tarea 1: Integración de API

- Utiliza los siguientes endpoints de la API JSONPlaceholder:
  - [Usuarios](https://jsonplaceholder.typicode.com/users)
  - [Álbumes](https://jsonplaceholder.typicode.com/albums)
  - [Fotos](https://jsonplaceholder.typicode.com/photos)
- Transforma los datos JSON extraídos en un formato tabular.
- Agrega una nueva columna llamada `extraction_date` para registrar la fecha de extracción de los datos.
- Asegúrate de que todos los campos en los datos extraídos se guarden como tipo `STRING`.
- Guarda los datos en formato **Parquet**.
- Verifica el conteo de filas:
  - Tabla `Usuarios`: 10 registros
  - Tabla `Álbumes`: 100 registros
  - Tabla `Fotos`: 5000 registros.


In [2]:
import pandas as pd
import numpy as np
import requests
import pyarrow

In [3]:
import requests
# APIs
url_usuarios = 'https://jsonplaceholder.typicode.com/users'
url_albums = 'https://jsonplaceholder.typicode.com/albums'
url_fotos = 'https://jsonplaceholder.typicode.com/photos'

response1 = requests.get(url_usuarios)
response2 = requests.get(url_albums)
response3 = requests.get(url_fotos)

In [5]:
df_usuarios = pd.DataFrame(response1.json())
df_albums = pd.DataFrame(response2.json())
df_fotos = pd.DataFrame(response3.json())

In [6]:
# Nuea columna

df_usuarios['extraction_date'] =   "2025-06-06"
df_albums['extraction_date'] =  "2025-06-06"
df_fotos['extraction_date'] =  "2025-06-06"

In [12]:
df_usuarios.dtypes

,0
id,int64
name,object
username,object
email,object
address,object
phone,object
website,object
company,object
extraction_date,object


In [13]:
### Aseguramos que los tipos de datos extraídos sean STRING > OBJECT
df_albums.dtypes

,0
userId,int64
id,int64
title,object
extraction_date,object


In [14]:
### Aseguramos que los tipos de datos extraídos sean STRING > OBJECT
df_fotos.dtypes

,0
albumId,int64
id,int64
title,object
url,object
thumbnailUrl,object
extraction_date,object


In [7]:
# Guardar en formato parquet

df_usuarios.to_parquet('df_usuarios.parquet.gzip', compression='gzip')
df_albums.to_parquet('df_albums.parquet.gzip', compression='gzip')
df_fotos.to_parquet('df_fotos.parquet.gzip', compression='gzip')

In [8]:
# Verificar conteo de filas
df_usuarios.shape


(100, 4)

In [9]:
df_albums.shape

(100, 4)

In [10]:
df_fotos.shape

(5000, 6)

## Tarea 2: Tarea de Transformación

### Notas:
Utiliza el archivo Parquet generado a partir de la tabla `Usuarios` en el paso de extracción.

### Descripción:
La columna `phone` en la tabla `Usuarios` contiene formatos inconsistentes. Limpia y transforma esta columna aplicando las siguientes reglas:

- **Regla 1**: Elimina todos los caracteres especiales del número de teléfono para que solo queden caracteres numéricos.  
  - **Ejemplo**: `210.067.6132 → 2100676132`
- **Regla 2**: Si un número de teléfono incluye una extensión (indicada por `x`), separa la extensión en una nueva columna llamada `ext`.  
  - **Ejemplo**: `586.493.6943 x140 → phone: 5864936943, ext: 140`

Guarda los datos limpios en un nuevo archivo Parquet.

### Criterios de aceptación:
- La columna `phone` está libre de caracteres especiales.
- Las extensiones se separan correctamente en la columna `ext`.
- La tabla final limpia se guarda en formato Parquet.

In [15]:
# Leer archivos parquet

pq_usuarios = pd.read_parquet('df_usuarios.parquet.gzip')
pq_albums = pd.read_parquet('df_albums.parquet.gzip')
pq_fotos = pd.read_parquet('df_fotos.parquet.gzip')

In [16]:
# Copiamos y asignamos a nuevos dfs
usuarios = pq_usuarios.copy()
usuarios['phone'].head(11)

,phone
0,1-770-736-8031 x56442
1,010-692-6593 x09125
2,1-463-123-4447
3,493-170-9623 x156
4,(254)954-1289
5,1-477-935-8478 x6430
6,210.067.6132
7,586.493.6943 x140
8,(775)976-6794 x41206
9,024-648-3804


In [17]:
#  Limpia y transforma
usuarios['phone'] =  usuarios['phone'].str.replace('-', '')
usuarios['phone'] =  usuarios['phone'].str.replace('.', '')
usuarios['phone'] =  usuarios['phone'].str.replace('(', '')
usuarios['phone'] =  usuarios['phone'].str.replace(')', '')
usuarios['phone'] =  usuarios['phone'].str.replace('x', 'ext:')
usuarios['phone'] =  usuarios['phone'].str.replace(' ', ',')

In [19]:
# Resultado limpio
usuarios['phone'].head(11)

,phone
0,"17707368031,ext:56442"
1,"0106926593,ext:09125"
2,14631234447
3,"4931709623,ext:156"
4,2549541289
5,"14779358478,ext:6430"
6,2100676132
7,"5864936943,ext:140"
8,"7759766794,ext:41206"
9,0246483804


In [23]:
# Guardamos de nuevo  en formato parquet

usuarios.to_parquet('users_cleaned.parquet.gzip', compression='gzip')


## Tarea 3: Cargar Datos en una Solución en la Nube

### Descripción:
Como Ingeniero de Datos Senior, se te asigna la tarea de cargar la tabla `users_df` limpia del paso de transformación en una solución de almacenamiento de datos basada en la nube, específicamente Google BigQuery.

### Criterios de aceptación:
- Escribe pseudocódigo para cargar el archivo `users_cleaned.parquet` en una tabla de BigQuery.
- Incluye comentarios detallados que expliquen:
  - Los pasos necesarios para configurar el acceso a BigQuery (por ejemplo, configuración de credenciales de cuenta de servicio).
  - Cómo crear un conjunto de datos y una tabla en BigQuery.
- El pseudocódigo debe incluir:
  - Cargar el archivo Parquet en una tabla de BigQuery.
  - Verificar la carga imprimiendo un mensaje de éxito o manejando errores.


In [24]:
# ver adjunto

In [ ]:
# Pseudocódigo sencillo para cargar users_cleaned.parquet a BigQuery

1. CONFIGURAR CREDENCIALES
   • Definir ruta al archivo de credenciales JSON de la cuenta de servicio:
     GOOGLE_APPLICATION_CREDENTIALS = "/00034/a/bigquery-service-account.json"
   • (En el entorno) Exportar variable de entorno:
     export GOOGLE_APPLICATION_CREDENTIALS="/00034/a/bigquery-service-account.json"

2. INICIALIZAR CLIENTE DE BIGQUERY
   • CLIENTE ← bigquery.Client(PROJECT_ID = "Copia de Copia de prueba_espaniol_de")

3. VERIFICAR O CREAR DATASET
   • DATASET_REF ← CLIENTE.dataset("analytics_dataset")
   • Intentar:
       CLIENTE.get_dataset(DATASET_REF)
       → Si existe, imprimir "Dataset existe"
     Capturar NotFound:
       DATASET_OBJ ← bigquery.Dataset(DATASET_REF)
       DATASET_OBJ.location ← "US"
       CLIENTE.create_dataset(DATASET_OBJ)
       → Imprimir "Dataset creado"

4. PREPARAR REFERENCIA A LA TABLA
   • TABLE_REF ← DATASET_REF.table("users_cleaned")

5. CONFIGURAR JOB DE CARGA
   • JOB_CONFIG ← new LoadJobConfig()
   • JOB_CONFIG.source_format ← PARQUET
   • JOB_CONFIG.write_disposition ← WRITE_TRUNCATE
   • (Opcional) Definir esquema manual en JOB_CONFIG.schema

6. EJECUTAR JOB DE CARGA
   Si PARQUET_URI comienza con "gs://":
     • LOAD_JOB ← CLIENTE.load_table_from_uri(
         source_uri = PARQUET_URI,
         destination = TABLE_REF,
         job_config = JOB_CONFIG
       )
   Sino:
     • Abrir archivo local en modo binario:
         ARCHIVO ← abrir(PARQUET_URI, "rb")
       LOAD_JOB ← CLIENTE.load_table_from_file(
         file_obj = ARCHIVO,
         destination = TABLE_REF,
         job_config = JOB_CONFIG
       )
   • Imprimir "Carga iniciada. ID del job: " + LOAD_JOB.job_id
   • Esperar a que LOAD_JOB termine (por ejemplo, LOAD_JOB.result(timeout=600))
     Capturar excepciones:
       → Imprimir "Error en carga: " + mensaje de error
       → Terminar proceso

7. VERIFICAR RESULTADO (OPCIONAL)
   • TABLA ← CLIENTE.get_table(TABLE_REF)
   • Imprimir "Número de registros en tablas: " + TABLA.num_rows

8. FINALIZAR
   • Imprimir "Proceso de carga completado con éxito"



